TITANIC DATASET - ML - RANDOM FOREST
====================================

In [1]:
%load_ext autoreload
%autoreload 2

import mlflow
import pickle
import pandas as pd 
import numpy as np

from datetime import datetime
from functions import get_metrics

from matplotlib import pyplot as plt 
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

## Configs

In [2]:
EXPERIMENT_TITLE = "Titanic Dataset Analyzes"

run_description = """
### Descrição

Implementação usando [Random Forest](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
"""

tags = {
    "date": datetime.now(),
    "author": "Pablo Veinberg",
    "version": 1.0,
    "envoronment": "local",
    "mlflow.note.content": run_description,
    "mlflow.runName": "Random Forest",
    "data_source": "./../datasets/silver/train-encoded-not-normalize.parquet",
    "train_test_dataset": "./../datasets/silver/titanic-train-test-data.pkl"
}

params = {
    "token": np.random.randint(10_000,high=99_000),
    "criterion": "entropy"
    }


In [3]:
dataset = pd.read_parquet(tags['data_source'])
mlflow_dataset = mlflow.data.from_pandas(dataset, \
                                            source=tags['data_source'], \
                                            name="Titanic Dataset")

/home/pablo_veinberg/.local/lib/python3.10/site-packages/mlflow/data/dataset_source_registry.py:143: UserWarning: The specified dataset source can be interpreted in multiple ways: LocalArtifactDatasetSource, LocalArtifactDatasetSource. MLflow will assume that this is a LocalArtifactDatasetSource source.
  return _dataset_source_registry.resolve(


In [4]:
with open('./../datasets/silver/titanic-train-test-data.pkl', 'rb') as file:
    X_train, X_test, y_train, y_test = pickle.load(file)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((712, 8), (179, 8), (712,), (179,))

## Process

In [5]:
mlflow.set_experiment(EXPERIMENT_TITLE)

with mlflow.start_run():

    # Settings
    mlflow.log_input(mlflow_dataset)
    mlflow.set_tags(tags)
    mlflow.log_params(params)

    model = RandomForestClassifier(criterion=params["criterion"])
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    plot_confusion_matrix_path = f"./../results/random_forest_{params['token']}.png"
    metrics = get_metrics(y_test, y_pred, plot_confusion_matrix_path, "Matrix Confusion - Titanic Dataset")
    
    mlflow.log_metrics(metrics)
    mlflow.log_artifact(plot_confusion_matrix_path)
    
mlflow.end_run()

/home/pablo_veinberg/.local/lib/python3.10/site-packages/mlflow/data/pandas_dataset.py:134: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  return _infer_schema(self._df)


In [11]:
# Survival|0 = No, 1 = Yes
cm = confusion_matrix(y_test, y_pred)
cm

array([[102,   3],
       [  7,  67]])

In [76]:
class_report = classification_report(y_test, y_pred, 
                           output_dict=True)

results = class_report['macro avg']


# precisão indica quão bom o modelo é para detectar positivos

labels = ['não sobreviventes', 'sobreviventes']

for key, value in enumerate(labels):
    print(f"O modelo é bom em {round(class_report[str(key)]['precision'] * 100 ,2)}% para acertar {value}")
    print(f"Quando acerta, está correto num {round(class_report[str(key)]['recall'] * 100 ,2)}%")
    print(f"Esta informação foi baseada em {round(class_report[str(key)]['support'],0)} registros")
    print("="*50)

# print(f"O modelo é bom em {round(class_report['0']['precision'] * 100 ,2)}% para acertar não sobreviventes")
# print(f"Quando acerta, está correto num {round(class_report['0']['recall'] * 100 ,2)}%")


# # recall indica quando a resposta real é sim, quanto o modelo acertou
# print(class_report)

O modelo é bom em 93.58% para acertar não sobreviventes
Quando acerta, está correto num 97.14%
Esta informação foi baseada em 105 registros
O modelo é bom em 95.71% para acertar sobreviventes
Quando acerta, está correto num 90.54%
Esta informação foi baseada em 74 registros


In [79]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.97      0.95       105
           1       0.96      0.91      0.93        74

    accuracy                           0.94       179
   macro avg       0.95      0.94      0.94       179
weighted avg       0.94      0.94      0.94       179



(169, 10, 0.9441340782122905)